### Market Share Problem

In [2]:
import os
import json
import numpy as np
from docplex.mp.model import Model

In [3]:
# Setting random seed
seed = 0  # change this to generate different random instances
np.random.seed(seed)
num_products = 2 # Number of products (m) this is the only parameter that needs to be changed
# Define the folder to save all output files
output_folder = f"{num_products}_{seed}_market_sharing_results"
os.makedirs(output_folder, exist_ok=True)  # Create the folder if it doesn't exist

def create_market_sharing_model(num_products):
    """
    Creates a CPLEX model for the market-sharing problem with a 50/50 target split.

    Parameters:
    - num_products: int, number of products (m)

    Returns:
    - model: CPLEX model
    - demands: 2D list of int, demand matrix where demands[i][j] is the demand of retailer j for product i
    - target_demands: list of int, target demand for each product
    - x, s_plus, s_minus: CPLEX variables for retailer assignments and deviations
    """
    # Define the number of retailers based on the number of products
    num_retailers = 10 * (num_products - 1)

    # Generate a random demand matrix with values between 0 and 99
    demands = np.random.randint(0, 100, size=(num_products, num_retailers))

    # Calculate target demands (b_i) as half of the total demand per product, rounded down
    target_demands = [int(0.5 * sum(demands[i])) for i in range(num_products)]

    # Create a new model
    model = Model(name="Market Sharing Problem")
    model.set_time_limit(3600)  # Set a time limit of 1 hour (3600 seconds)
    
    # Variables
    x = model.binary_var_list(num_retailers, name="x")  # Binary variable for each retailer's assignment
    s_plus = model.continuous_var_list(num_products, name="s_plus", lb=0)  # Positive deviation
    s_minus = model.continuous_var_list(num_products, name="s_minus", lb=0)  # Negative deviation

    # Objective: Minimize sum of deviations from target split
    model.minimize(model.sum(s_plus[i] + s_minus[i] for i in range(num_products)))
    
    # Constraints
    for i in range(num_products):
        # Add constraint for product i to achieve the target split with deviation
        model.add_constraint(
            sum(demands[i][j] * x[j] for j in range(num_retailers)) + s_plus[i] - s_minus[i] == target_demands[i],
            f"demand_constraint_{i}"
        )

    return model, demands, target_demands, x, s_plus, s_minus


In [4]:
# Create the model
model, demands, target_demands, x, s_plus, s_minus = create_market_sharing_model(num_products)

print(model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Market Sharing Problem

Minimize
 obj: s_plus_0 + s_plus_1 + s_minus_0 + s_minus_1
Subject To
 demand_constraint_0: 44 x_0 + 47 x_1 + 64 x_2 + 67 x_3 + 67 x_4 + 9 x_5
                      + 83 x_6 + 21 x_7 + 36 x_8 + 87 x_9 + s_plus_0 - s_minus_0
                       = 262
 demand_constraint_1: 70 x_0 + 88 x_1 + 88 x_2 + 12 x_3 + 58 x_4 + 65 x_5
                      + 39 x_6 + 87 x_7 + 46 x_8 + 88 x_9 + s_plus_1 - s_minus_1
                       = 320

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9
End



In [5]:

def save_solution_to_json(filename, solution_data):
    """
    Saves the solution data to a JSON file.

    Parameters:
    - filename: str, name of the JSON file
    - solution_data: dict, dictionary containing solution details
    """
    with open(filename, 'w') as f:
        json.dump(solution_data, f, indent=4)

def save_solution_to_text(filename, solution, x, s_plus, s_minus, num_retailers, num_products):
    """
    Saves the solution data to a text file in the specified format.

    Parameters:
    - filename: str, name of the text file
    - solution: solution object
    - x: list of binary variables for retailer assignment
    - s_plus, s_minus: lists of continuous variables for deviations
    - num_retailers: int, number of retailers
    - num_products: int, number of products
    """
    with open(filename, 'w') as f:
        # Write objective value
        f.write(f"Objective Value (Sum of Deviations): {solution.objective_value}\n")
        
        # Write retailer assignments
        for i in range(num_retailers):
            division = "D1" if solution[x[i]] == 1 else "D2"
            f.write(f"Retailer {i + 1} assigned to {division}\n")
        
        # Write s_plus and s_minus values
        for i in range(num_products):
            f.write(f"s_plus[{i + 1}] = {solution[s_plus[i]]}\n")
            f.write(f"s_minus[{i + 1}] = {solution[s_minus[i]]}\n")





In [6]:

log_file = os.path.join(output_folder, "market_sharing_problem_log.txt")
with open(log_file, "w") as log_output:
    model.context.solver.log_output = log_output

    # Solve the model
    solution = model.solve()

# Prepare solution data for JSON
solution_data = {
    "objective_value": None,
    "retailer_assignments": [],
    "s_plus_values": [],
    "s_minus_values": [],
    "demands": demands.tolist(),
    "target_demands": target_demands
}

# Check if a solution is found
if solution:
    solution_data["objective_value"] = solution.objective_value
    for i in range(10 * (num_products - 1)):
        solution_data["retailer_assignments"].append({
            "retailer": i + 1,
            "assigned_to": "D1" if solution[x[i]] == 1 else "D2"
        })
    for i in range(num_products):
        solution_data["s_plus_values"].append(solution[s_plus[i]])
        solution_data["s_minus_values"].append(solution[s_minus[i]])

    # Save solution to text file in the specified format
    result_file = os.path.join(output_folder, "market_sharing_result.txt")
    save_solution_to_text(result_file, solution, x, s_plus, s_minus, 10 * (num_products - 1), num_products)
else:
    solution_data["status"] = "No feasible solution found."

# Save solution to JSON file
json_file = os.path.join(output_folder, "market_sharing_solution.json")
save_solution_to_json(json_file, solution_data)

print(f"Solution saved to {json_file}")
print(f"Formatted result saved to {result_file}")
print(f"Log saved to {log_file}")


Solution saved to 2_0_market_sharing_results\market_sharing_solution.json
Formatted result saved to 2_0_market_sharing_results\market_sharing_result.txt
Log saved to 2_0_market_sharing_results\market_sharing_problem_log.txt
